In [1]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.parse import DependencyGraph
from pymorphy2.tokenizers import simple_word_tokenize

In [2]:
m = MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

# Задание 1

Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).

In [3]:
!C:\Users\user\Desktop\Aot\hw2\udpipe-1.2.0-bin\bin-win64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\user\Desktop\Aot\hw2\russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > corpus.conllu

Loading UDPipe model: done.


In [65]:
trees = []

with open('corpus.conllu', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [66]:
print(trees[1])

1	И	и	CCONJ	_	_	6	cc	_	_
2	тогда	тогда	ADV	_	Degree=Pos	6	advmod	_	_
3	о	о	ADP	_	_	4	case	_	_
4	разделении	разделений	NOUN	_	Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing	6	obl	_	_
5	властей	власть	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	4	nmod	_	_
6	придется	прийтись	VERB	_	Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Fut|VerbForm=Fin|Voice=Mid	0	root	_	_
7	окончательно	окончательно	ADV	_	Degree=Pos	8	advmod	_	_
8	забыть	забыть	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	6	csubj	_	SpaceAfter=No
9	.	.	PUNCT	_	_	6	punct	_	SpacesAfter=\r\n\t


In [67]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    return ' '.join(lemmas)

In [68]:
verbs_50 = []
for tree in trees:
    for line in tree.split('\n'):
        elements = line.split('\t')
        if len(elements) > 3 and elements[3] == 'VERB':
            #print(elements)
            verbs_50.append(elements[2])
print(len(verbs_50), verbs_50[:5])
verbs_c = Counter(verbs_50)
print(len(verbs_c))
verbs_50 = [x[0] for x in verbs_c.items() if x[1] >= 50]
print(len(verbs_50), verbs_50[:5])

17055 ['предложить', 'объединить', 'заявить', 'прийтись', 'забыть']
2120
65 ['заявить', 'подать', 'просить', 'признать', 'стать']


In [69]:
groups = []
for i, tree in enumerate(trees):
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        for element in g.triples():
                #print(element)
                #print(element[1])
                #break
            if element[0][1] == 'VERB' and element[1] == 'obj' and element[2][1] == 'NOUN':
                norm_el = normalize(element[0][0])
                if norm_el in verbs_50:
                    groups.append((normalize(element[0][0]), normalize(element[2][0])))
    except:
        pass
    #break

In [70]:
groups[:5]

[('подать', 'иск'),
 ('признать', 'договор'),
 ('рассмотреть', 'вопрос'),
 ('обвинить', 'руководство'),
 ('получить', 'срок')]

In [71]:
len(groups)

1974

# Задание 2

Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).

In [12]:
import nltk
from nltk.collocations import *

In [72]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(groups)

In [73]:
finder.apply_freq_filter(3)

In [74]:
likelihood = finder.nbest(nltk.collocations.BigramAssocMeasures().likelihood_ratio, 100)
dice = finder.nbest(nltk.collocations.BigramAssocMeasures().dice, 100)
pmi = finder.nbest(nltk.collocations.BigramAssocMeasures().pmi, 100)

In [75]:
likelihood[:10]

[('подать', 'иск'),
 ('принять', 'решение'),
 ('удовлетворить', 'иск'),
 ('вынести', 'приговор'),
 ('просить', 'суд'),
 ('обжаловать', 'решение'),
 ('вынести', 'решение'),
 ('удовлетворить', 'ходатайство'),
 ('отклонить', 'жалоба'),
 ('предъявить', 'обвинение')]

In [76]:
likeli_scores = finder.score_ngrams(nltk.collocations.BigramAssocMeasures().likelihood_ratio)
dice_scores = finder.score_ngrams(nltk.collocations.BigramAssocMeasures().dice)
pmi_scores = finder.score_ngrams(nltk.collocations.BigramAssocMeasures().pmi)

In [77]:
likeli_scores[:10]

[(('подать', 'иск'), 373.0076053454368),
 (('принять', 'решение'), 199.8617926261594),
 (('удовлетворить', 'иск'), 174.50888096556136),
 (('вынести', 'приговор'), 173.35989706128592),
 (('просить', 'суд'), 170.63331645005636),
 (('обжаловать', 'решение'), 163.51914253223453),
 (('вынести', 'решение'), 121.10258287873937),
 (('удовлетворить', 'ходатайство'), 120.93951872963609),
 (('отклонить', 'жалоба'), 100.35520276852321),
 (('предъявить', 'обвинение'), 97.12141507085039)]

In [78]:
dice_scores[:10]

[(('пройти', 'прение'), 0.6666666666666666),
 (('решить', 'проблема'), 0.6),
 (('запретить', 'деятельность'), 0.5833333333333334),
 (('просить', 'суд'), 0.5555555555555556),
 (('подать', 'иск'), 0.5088757396449705),
 (('делать', 'вывод'), 0.46153846153846156),
 (('назначить', 'наказание'), 0.46153846153846156),
 (('выплатить', 'компенсация'), 0.45714285714285713),
 (('заявить', 'отвод'), 0.4444444444444444),
 (('предъявить', 'обвинение'), 0.43333333333333335)]

In [79]:
pmi_scores[:10]

[(('решить', 'проблема'), 9.139551352398794),
 (('назначить', 'наказание'), 8.624978179569034),
 (('объявить', 'голодовка'), 8.361943773735241),
 (('делать', 'вывод'), 8.209940680290192),
 (('заявить', 'отвод'), 8.139551352398794),
 (('пройти', 'прение'), 8.139551352398794),
 (('запретить', 'деятельность'), 7.677445599463171),
 (('доказать', 'незаконность'), 7.6249781795690375),
 (('обязать', 'прокуратура'), 7.139551352398794),
 (('выплатить', 'штраф'), 6.969626350956482)]

# Задание 3

Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.

In [80]:
top_over_metrics = set(pmi) & set(dice) & set(likelihood)
print(len(top_over_metrics))
top_over_metrics

78


{('взыскать', 'деньга'),
 ('взыскать', 'компенсация'),
 ('вынести', 'вердикт'),
 ('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('дать', 'возможность'),
 ('дать', 'киллер'),
 ('дать', 'показание'),
 ('дать', 'санкция'),
 ('дать', 'свет'),
 ('дать', 'согласие'),
 ('делать', 'вывод'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('заявить', 'ходатайство'),
 ('иметь', 'место'),
 ('иметь', 'право'),
 ('иметь', 'шанс'),
 ('использовать', 'знак'),
 ('использовать', 'положение'),
 ('использовать', 'схема'),
 ('назначить', 'наказание'),
 ('направить', 'дело'),
 ('направить', 'письмо'),
 ('направить', 'представление'),
 ('обвинить', 'глава'),
 ('обвинить', 'господин'),
 ('обвинить', 'компания'),
 ('обвинить', 'президент'),
 ('обжаловать', 'постановление'),
 ('обжаловать', 'решение'),
 ('объявить', 'голодовка'),
 ('обязать', 'прокуратура'),
 ('оспаривать', 'зак

In [81]:
with open('verb_coll.txt', encoding='utf-8') as f:
    lines = f.readlines()

verb_coll = []
for line in lines:
    element = normalize(line.split('\t')[-1]).split() 
    verb_coll.append(tuple(element))

In [82]:
golden_standart = set(verb_coll) & top_over_metrics
len(golden_standart)

27

In [83]:
golden_standart

{('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('дать', 'возможность'),
 ('дать', 'показание'),
 ('дать', 'санкция'),
 ('дать', 'согласие'),
 ('делать', 'вывод'),
 ('заявить', 'ходатайство'),
 ('иметь', 'шанс'),
 ('направить', 'дело'),
 ('обжаловать', 'решение'),
 ('отменить', 'постановление'),
 ('передать', 'дело'),
 ('подать', 'иск'),
 ('получить', 'срок'),
 ('предъявить', 'обвинение'),
 ('предъявить', 'претензия'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('провести', 'обыск'),
 ('провести', 'проверка'),
 ('рассматривать', 'дело'),
 ('рассмотреть', 'вопрос'),
 ('рассмотреть', 'жалоба'),
 ('решить', 'проблема'),
 ('удовлетворить', 'требование')}

# Задание 4

Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.

In [84]:
top_over_metrics - golden_standart

{('взыскать', 'деньга'),
 ('взыскать', 'компенсация'),
 ('вынести', 'вердикт'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('дать', 'киллер'),
 ('дать', 'свет'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('иметь', 'место'),
 ('иметь', 'право'),
 ('использовать', 'знак'),
 ('использовать', 'положение'),
 ('использовать', 'схема'),
 ('назначить', 'наказание'),
 ('направить', 'письмо'),
 ('направить', 'представление'),
 ('обвинить', 'глава'),
 ('обвинить', 'господин'),
 ('обвинить', 'компания'),
 ('обвинить', 'президент'),
 ('обжаловать', 'постановление'),
 ('объявить', 'голодовка'),
 ('обязать', 'прокуратура'),
 ('оспаривать', 'законность'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'предписание'),
 ('оспорить', 'претензия'),
 ('оспорить', 'сделка'),
 ('оставить', 'приговор'),
 ('отказать', 'жалоба'),
 ('отклонить', 'жалоба'),
 ('отменить', 'регистрация'),
 ('отменить', 'указ'),
 ('подать', 'апелляция'),
 ('подтвердить', 'зако

In [85]:
to_add = [('вынести', 'вердикт'), ('выплатить', 'штраф'), ('иметь', 'право'), ('обжаловать', 'постановление'), ('подать', 'апелляция'), ('рассмотреть', 'ходатайство'),
('удовлетворить', 'ходатайство')]

Я бы добавила эти коллокации, т.к. они, как мне кажется, часто употребляемые, в целом разумные (в отличие от всяких "дать лицо") и они тот факт, что они попали в пересечение трех метрих - тоже можно считать причиной добавления